目的
---
ナップザック問題をカスタムクラスを使って解いてみる

参考
---
- https://prog.suke-blog.jp/entry/2018/06/30/170549
- https://prog.suke-blog.jp/entry/2018/08/27/170537

In [13]:
from platypus import NSGAII, Type, Problem,Constraint, Binary, nondominated, unique
import pandas as pd
import random

# 決定変数(Type)の定義

- 決定変数はTypeを継承することで独自定義する
- (サンプルで使われるRealもTypeを継承したクラス)

In [14]:
# サンプルデータ
# 価値と重さの組み合わせ配列
df_items = pd.read_csv('./files/items.csv')
df_items.head()

,価値,重さ
0,32,20
1,52,80
2,69,49
3,42,95
4,19,68


In [57]:
items = 7
capacity = 9
weights = [2, 3, 6, 7, 5, 9, 4]
values = [6, 5, 8, 9, 6, 7, 3]


Problemの定義
---


In [56]:
class Knapsack(Problem):
    """問題定義のクラス
    """
    def __init__(self, values, weights, capacity):
        """
        https://github.com/Project-Platypus/Platypus/blob/master/platypus/core.py#L93
        Problemはコンストラクタで
        1: 決定変数の数
        2: 目的変数の数
        3: (option) 制約の数
        4: (option) 目的関数
        を受け取る
        """
        super(Knapsack, self).__init__(1, 1, 1)
        # Binaryは0:1(false:true)
        self.types[:] = Binary(len(values))
        self.directions[0] = Problem.MAXIMIZE
        self.constraints[0] = Constraint("<=", capacity)
        
    def evaluate(self, solution):
        selection = solution.variables[0]
        total_weight = sum([weights[i] if selection[i] else 0 for i in range(items)])
        total_profit = sum([values[i] if selection[i] else 0 for i in range(items)])
        solution.objectives = [total_profit]
        solution.constraints = [total_weight]

In [55]:
items = 7
capacity = 9.0
weights = [2, 3, 6, 7, 5, 9, 4]
values = [6, 5, 8, 9, 6, 7, 3]

knapsack = Knapsack(values, weights, capacity)
algorithm = NSGAII(knapsack)
algorithm.run(100)

# nondominated(algorithm.result)
# 支配的でない解をフィルタリングする(支配的？ってどういう意味?)
# unique(objectives)
# 同じ解を除外する
for solution in unique(nondominated(algorithm.result)):
    print(solution.variables, solution.objectives)

[[True, False, False, True, False, False, False]] [15]


In [54]:
from platypus import GeneticAlgorithm, Problem, Constraint, Binary, nondominated, unique

# This simple example has an optimal value of 15 when picking items 1 and 4.
items = 7
capacity = 9
weights = [2, 3, 6, 7, 5, 9, 4]
profits = [6, 5, 8, 9, 6, 7, 3]
    
def knapsack(x):
    selection = x[0]
    total_weight = sum([weights[i] if selection[i] else 0 for i in range(items)])
    total_profit = sum([profits[i] if selection[i] else 0 for i in range(items)])
    return total_profit, total_weight

problem = Problem(1, 1, 1)
problem.types[0] = Binary(items)
problem.directions[0] = Problem.MAXIMIZE
problem.constraints[0] = Constraint("<=", capacity)
problem.function = knapsack

algorithm = NSGAII(problem)
algorithm.run(10000)

for solution in unique(nondominated(algorithm.result)):
    print(solution.variables, solution.objectives)

[[True, False, False, True, False, False, False]] [15]
